<a href="https://colab.research.google.com/github/Sachin1801/Trading-Bot/blob/main/Trading_Bot_Oanda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1 - Import test data

In [ ]:
!pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=09d35679bfe3cad7c268f9a8de3db0b377c01f7004368ac30de201a0adfca0ae
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [ ]:
import yfinance as yf
import pandas_ta as pa
import plotly.graph_objects as go
import numpy as np
from datetime import datetime
import datetime
import pandas as pd

def get_data(symbol: str):
     end_date = datetime.date.today()
     start_date = end_date - datetime.timedelta(days=1000)
     data = yf.download(tickers=symbol, start=start_date, end=end_date, interval='1d')
     return data
# Get the data
data = get_data('AAPL')

[*********************100%***********************]  1 of 1 completed


In [ ]:
data


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-01-12,176.119995,177.179993,174.820007,175.529999,172.891022,74805200
2022-01-13,175.779999,176.619995,171.789993,172.190002,169.601257,84505800
2022-01-14,171.339996,173.779999,171.089996,173.070007,170.468048,80440800
2022-01-18,171.509995,172.539993,169.410004,169.800003,167.247192,90956700
2022-01-19,170.000000,171.080002,165.940002,166.229996,163.730865,94815000
...,...,...,...,...,...,...
2024-10-01,229.520004,229.649994,223.740005,226.210007,226.210007,63285000
2024-10-02,225.889999,227.369995,223.020004,226.779999,226.779999,32880600
2024-10-03,225.139999,226.809998,223.320007,225.669998,225.669998,34044200


In [ ]:
import plotly.graph_objects as go
dfpl=data[:]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                    open=dfpl['Open'],
                    high=dfpl['High'],
                    low=dfpl['Low'],
                    close=dfpl['Close'])])

fig.update_layout(
    autosize=False,
    width=1000,
    height=800,
    paper_bgcolor='white',
    plot_bgcolor='white')
fig.update_xaxes(gridcolor='black')
fig.update_yaxes(gridcolor='black')

fig.show()

## 2 - Signals functions

In [ ]:
def identify_rejection(data):
    # Create a new column for rejection signal
    data['rejection'] = data.apply(lambda row: 2 if ( #for bullish pattern we are returning 2
        ( (min(row['Open'], row['Close']) - row['Low']) > (1.5 * abs(row['Close'] - row['Open']))) and
        (row['High'] - max(row['Close'], row['Open'])) < (0.8 * abs(row['Close'] - row['Open'])) and
        (abs(row['Open'] - row['Close']) > row['Open'] * 0.001)
    ) else 1 if ( # this is to identify bearish patterns
        (row['High'] - max(row['Open'], row['Close'])) > (1.5 * abs(row['Open'] - row['Close'])) and
        (min(row['Close'], row['Open']) - row['Low']) < (0.8 * abs(row['Open'] - row['Close'])) and
        (abs(row['Open'] - row['Close']) > row['Open'] * 0.001)
    ) else 0, axis=1) #no rejection pattern

    return data
data = identify_rejection(data)


In [ ]:
def identify_rejection(data):
    # Create a new column for rejection signal
    data['rejection'] = data.apply(lambda row: 2 if ( #for bullish pattern we are returning 2
        ( (min(row['Open'], row['Close']) - row['Low']) > (1.5 * abs(row['Close'] - row['Open']))) and
        (row['High'] - max(row['Close'], row['Open'])) < (0.8 * abs(row['Close'] - row['Open'])) and
        (abs(row['Open'] - row['Close']) > row['Open'] * 0.001)
    ) else 1 if ( # this is to identify bearish patterns
        (row['High'] - max(row['Open'], row['Close'])) > (1.5 * abs(row['Open'] - row['Close'])) and
        (min(row['Close'], row['Open']) - row['Low']) < (0.8 * abs(row['Open'] - row['Close'])) and
        (abs(row['Open'] - row['Close']) > row['Open'] * 0.001)
    ) else 0, axis=1) #no rejection pattern

    return data

def pointpos(x, xsignal): #is rejection
    if x[xsignal]==1: #if rejection col
        return x['High']+1e-4
    elif x[xsignal]==2:
        return x['Low']-1e-4
    else:
        return np.nan

def plot_with_signal(dfpl):

    fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                    open=dfpl['Open'],
                    high=dfpl['High'],
                    low=dfpl['Low'],
                    close=dfpl['Close'])])


    fig.update_layout(
        autosize=False,
        width=1000,
        height=800,
        paper_bgcolor='black',
        plot_bgcolor='black')
    fig.update_xaxes(gridcolor='black')
    fig.update_yaxes(gridcolor='black')
    fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                    marker=dict(size=8, color="MediumPurple"),
                    name="Signal")
    fig.show()



#condition for applying support level
def support(df1, l, n1, n2): #n1[variable name for before candle] n2(variable for aftr candle) before and after candle l(current candle)
    if ( df1.Low[l-n1:l].min() < df1.Low[l] or #if there is a candle on the right and left of current that has a LOW lower, then we return 0(false)
        df1.Low[l+1:l+n2+1].min() < df1.Low[l] ):
        return 0
    return 1 #this menas it is a support level

#same condition checking for the resistance
def resistance(df1, l, n1, n2): #n1 n2 before and after candle l
    if ( df1.High[l-n1:l].max() > df1.High[l] or
       df1.High[l+1:l+n2+1].max() > df1.High[l] ):
        return 0
    return 1

def closeResistance(l, levels, lim, df): #levels- list of resistance levels, lim- threshhold for limit if falls in level or not
    if len(levels) == 0: #if no levels there is nothing to test
        return 0
    #condtions to check if the candle is even close enough to any level of resistance
    c1 = abs(df['High'][l] - min(levels, key=lambda x: abs(x - df['High'][l]))) <= lim #condition 1
    c2 = abs(max(df['Open'][l], df['Close'][l]) - min(levels, key=lambda x: abs(x - df['High'][l]))) <= lim #condition

    c3 = min(df['Open'][l], df['Close'][l]) < min(levels, key=lambda x: abs(x - df['High'][l]))
    c4 = df['Low'][l] < min(levels, key=lambda x: abs(x - df['High'][l]))
    if (c1 or c2) and c3 and c4:
        return min(levels, key=lambda x: abs(x - df['High'][l])) #this actually tells us that which level is the candle actually close to
    else:
        return 0

#this the counter part of resistance
def closeSupport(l, levels, lim, df):
    if len(levels) == 0:
        return 0
    c1 = abs(df['Low'][l] - min(levels, key=lambda x: abs(x - df['Low'][l]))) <= lim
    c2 = abs(min(df['Open'][l], df['Close'][l]) - min(levels, key=lambda x: abs(x - df['Low'][l]))) <= lim
    c3 = max(df['Open'][l], df['Close'][l]) > min(levels, key=lambda x: abs(x - df['Low'][l]))
    c4 = df['High'][l] > min(levels, key=lambda x: abs(x - df['Low'][l]))
    if (c1 or c2) and c3 and c4:
        return min(levels, key=lambda x: abs(x - df['Low'][l]))
    else:
        return 0

def is_below_resistance(l, level_backCandles, level, df):
    return df.loc[l-level_backCandles:l-1, 'High'].max() < level

def is_above_support(l, level_backCandles, level, df):
    return df.loc[l-level_backCandles:l-1, 'Low'].min() > level





## 2 - Define and test your signal function

In [ ]:
#l - index current candle
#n1,n2 - no. of preceeding and following neighbours
#levelbackCandles- large time window before the current candle(l)
#windowback - for smaller window frames

def check_candle_signal(l, n1, n2, levelbackCandles, windowbackCandles, df):
    df=identify_rejection(df)
    #list for support
    ss = []
    #list for resistance
    rr = []

    for subrow in range(l-levelbackCandles, l-n2+1): #small gap to avoid look-ahead bias
        #if support - append support vale in ss
        if support(df, subrow, n1, n2):
            ss.append(df.Low[subrow])
        #if resistance - append resistance vale in ss
        if resistance(df, subrow, n1, n2):
            rr.append(df.High[subrow])

    ss.sort() #keep lowest support when popping a level
    for i in range(1,len(ss)):
        if(i>=len(ss)):
            break
        if abs(ss[i]-ss[i-1])/ss[i]<=0.001: # merging close distance levels
            ss.pop(i)

    rr.sort(reverse=True) # keep highest resistance when popping one
    for i in range(1,len(rr)):
        if(i>=len(rr)):
            break
        if abs(rr[i]-rr[i-1])/rr[i]<=0.001: # merging close distance levels
            rr.pop(i)

    #----------------------------------------------------------------------
    # joined levels
    #making a combined list rrss for holding both resistance and support levels
    rrss = rr+ss
    rrss.sort()
    for i in range(1,len(rrss)):
        if(i>=len(rrss)):
            break
        if abs(rrss[i]-rrss[i-1])/rrss[i]<=0.001: # merging close distance levels
            rrss.pop(i)
    #test if the current candle is within a close percentage to resistance/support level
    cR = closeResistance(l, rrss, df.Close[l]*0.003, df)
    cS = closeSupport(l, rrss, df.Close[l]*0.003, df)
    #----------------------------------------------------------------------

    # cR = closeResistance(l, rr, 150e-5, df)
    # cS = closeSupport(l, ss, 150e-5, df)
    #print(rrss,df.Close*0.002)
    #condition that we are moving to a resistance level and we closeresis also is true and we also see a bearish candle
    if (df.rejection[l] == 1 and cR and is_below_resistance(l,windowbackCandles,cR, df)):
        return 1 #1 is a signal for bearsih
    elif(df.rejection[l] == 2 and cS and is_above_support(l,windowbackCandles,cS, df)):
        return 2 #2 is the signal for bullish
    else:
        return 0

#check_candle_signal(l=len(data)-1, n1=10, n2=10, levelbackCandles=200, windowbackCandles=11, df=data)

n1=5
n2=5
levelbackCandles=100
windowbackCandles= n2

signal = [0 for i in range(0, levelbackCandles + n1 + 1)] #fill 0 signals for first rows

for i in range(0, len(data) - 1 - levelbackCandles - n1):
    df = data[i + n1 + 1:i + levelbackCandles + n1 + 1 + 1].copy()
    df.reset_index(inplace=True)

    # Validate the return value of check_candle_signal before appending
    result = check_candle_signal(l=len(df)-1,
                                    n1=n1,
                                    n2=n2,
                                    levelbackCandles=levelbackCandles,
                                    windowbackCandles=windowbackCandles,
                                    df=df)
    signal.append(result)
data["signal"] = signal

In [ ]:
data.signal.value_counts()

,count
signal,
0,683
1,2
2,2


In [ ]:
data['pointpos'] = data.apply(lambda row: pointpos(row,"signal"), axis=1)
plot_with_signal(data[:])

### 3 - Connect to the market and execute trades

In [ ]:
from apscheduler.schedulers.blocking import BlockingScheduler
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
from oandapyV20.contrib.requests import MarketOrderRequest
from oanda_candles import Pair, Gran, CandleClient
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails

In [ ]:
from config import access_token, accountID
access_token='XXXXXXX'
accountID =
#to download the candle from online api
def get_candles(n):
    #access_token='XXXXXXX'#you need token here generated from OANDA account
    client = CandleClient(access_token, real=False)
    collector = client.get_collector(Pair.BTC_USD, Gran.D1)
    candles = collector.grab(n)
    return candles

candles = get_candles(300)
# for candle in candles:
#     print(float(str(candle.bid.o))>1)


True
True
True


In [ ]:
def trading_job():
    candles = get_candles(300)
    dfstream = pd.DataFrame(columns=['Open','Close','High','Low'])

    i=0
    for candle in candles:
        dfstream.loc[i, ['Open']] = float(str(candle.bid.o))
        dfstream.loc[i, ['Close']] = float(str(candle.bid.c))
        dfstream.loc[i, ['High']] = float(str(candle.bid.h))
        dfstream.loc[i, ['Low']] = float(str(candle.bid.l))
        i=i+1

    dfstream['Open'] = dfstream['Open'].astype(float)
    dfstream['Close'] = dfstream['Close'].astype(float)
    dfstream['High'] = dfstream['High'].astype(float)
    dfstream['Low'] = dfstream['Low'].astype(float)

    signal = check_candle_signal(l=len(dfstream)-1, n1=6, n2=6, levelbackCandles=200, windowbackCandles=7, df=dfstream)

    # EXECUTING ORDERS
    #accountID = "XXXXXXX" #your account ID here
    client = API(access_token)

    SLTPRatio = 2.

    SLBuy = dfstream['Low'].iloc[-1]
    SLSell = dfstream['High'].iloc[-1]

    TPBuy = dfstream['Close'].iloc[-1]+(dfstream['Close'].iloc[-1]-SLBuy)*SLTPRatio
    TPSell = dfstream['Close'].iloc[-1]-(dfstream['Close'].iloc[-1]-SLSell)*SLTPRatio

    print(dfstream.iloc[:-1,:])
    print(TPBuy, "  ", SLBuy, "  ", TPSell, "  ", SLSell)
    #signal = 2
    #Sell
    if signal == 1:
        mo = MarketOrderRequest(instrument="BTC_USD", units=-1, takeProfitOnFill=TakeProfitDetails(price=TPSell).data, stopLossOnFill=StopLossDetails(price=SLSell).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)
    #Buy
    elif signal == 2:
        mo = MarketOrderRequest(instrument="BTC_USD", units=1, takeProfitOnFill=TakeProfitDetails(price=TPBuy).data, stopLossOnFill=StopLossDetails(price=SLBuy).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)

### 4 - Executing orders automatically with a scheduler

In [ ]:
#trading_job()

scheduler = BlockingScheduler()
scheduler.add_job(trading_job, 'cron', hour='23', minute='55', start_date='2023-10-20 23:55:00', timezone='America/Chicago')
scheduler.start()